In [88]:
import plotly.express as px
import pandas as pd
import numpy as np

## Growth and Mortality - Assumptions

In this first example we're going to be following a single cohort - that is a single group of fish all born in the same year. In our example we'll be following "Fisheries Biology, Assessment, and Management"'s example of a cohort of cod (*Gadus morhua*). This cohort can be described by a couple of parameters:

1. Growth rate $K = 0.2/yr$
2. Maximum length $L_{\infty}= 100cm$
3. Natural instantaneous mortality $M=0.44/yr$
4. Weight per unit length $W=0.0082 g/cm^3$

Continuing to follow the example we start at age 2 with 1000 individuals of length 32.97.

The first question is how many survive each year. If $N(t)$ is the number of fish as a function of time we have:

$$\frac{dN}{dt}=-MN$$

whose solution is:

$$N=N_{0}e^{-Mt}$$

which in our case means $t$ is the number of years since age 2 and $N_{0}=1000$

The next question is how these cod are growing. In the example in the book they assume that growth is following a Von Bertalanffy growth curve:

$$L = L_{\infty}(1-e^{-Kt})$$

which is the solution to the differential equation:

$$ \frac{dD}{dt} = -kD, D=L-L_{\infty}$$

In [89]:
K = 0.2
L_max = 100
M = 0.44
W = 0.0082
N_0 = 1000
t_0 = 0 # for length

In [90]:
cod_df = pd.DataFrame(np.array(range(2, 13)), columns=['age'])
cod_df['length'] = round((L_max * (1 - np.exp(-K * (cod_df['age'] - t_0)))), 2)
cod_df['weight'] = (cod_df['length'] ** 3 * W).astype(int)
cod_df['number'] = (N_0 * np.exp(-M * (cod_df['age'] - 2))).astype(int)
cod_df['biomass'] = (cod_df['weight'] * cod_df['number'] / N_0).astype(int)
cod_df

,age,length,weight,number,biomass
0,2,32.97,293,1000,293
1,3,45.12,753,644,484
2,4,55.07,1369,414,566
3,5,63.21,2070,267,552
4,6,69.88,2798,172,481
5,7,75.34,3506,110,385
6,8,79.81,4168,71,295
7,9,83.47,4768,45,214
8,10,86.47,5301,29,153
9,11,88.92,5765,19,109


In [91]:
df = cod_df.copy()
df['weight'] = df['weight'] / df['weight'].max() * 100
df['biomass'] = df['biomass'] / df['biomass'].max() * 100
df['number'] = df['number'] / df['number'].max() * 100
px.line(df, x='age', y=['weight', 'biomass', 'number']).update_layout(
    yaxis_title="percent of max"
)

So what about fishing mortality $F$? Well this would just be added as part of the overall mortality.

$$N = N_{0}e^{-(M+F)t}$$

and then the catch can be estimated as:

$$C_t = \frac{F}{F+M} N_{0}(1 - e^{-(M+F)(t+1)}) - \frac{F}{F+M} N_{0}(1 - e^{-(M+F)t})$$

Note we're looking at how many will be left in the next year to understand how many are caught this year.

In the book they use another example for this which follows here:

In [92]:
K = 0.15
L_max = 45
M = 0.3
W = 0.0008
N_0 = 1000
t_0 = -0.44 # for length

def build_prawn_df(F):
    prawn_df = pd.DataFrame(np.array(range(4, 13)), columns=['age'])
    prawn_df['length'] = round((L_max * (1 - np.exp(-K * (prawn_df['age'] - t_0)))), 2)
    prawn_df['weight'] = round((prawn_df['length'] ** 3 * W), 2)
    prawn_df['number'] = round((N_0 * np.exp(-(M + F) * (prawn_df['age'] - 4))), 1)
    prawn_df['biomass'] = round((prawn_df['weight'] * prawn_df['number'] / N_0), 2)
    prawn_df['catch'] = round(F / (F + M) * N_0 * (np.exp(-(M + F) * (prawn_df['age'] - 4)) - np.exp(-(M + F) * (prawn_df['age'] - 4 + 1))), 2)
    prawn_df['catch_weight'] = round(prawn_df['catch'] * prawn_df['weight'], 2)
    return prawn_df

rows = []
for F in np.arange(0.1, 0.9 + 0.05, 0.05):
    df = build_prawn_df(F)
    rows.append({
        'F': F,
        'total_catch_weight': df['catch_weight'].sum()
    })
px.line(
    pd.DataFrame(rows),
    x='F', y='total_catch_weight'
)
    

In this example it seems that from a pure "weight of catch" perspective the higher the fishing mortality the better. However if we reduce the natural mortality rate we'll see something quite different.

In [93]:
M = 0.1

rows = []
for F in np.arange(0.1, 0.9 + 0.05, 0.05):
    df = build_prawn_df(F)
    rows.append({
        'F': F,
        'total_catch_weight': df['catch_weight'].sum()
    })
px.line(
    pd.DataFrame(rows),
    x='F', y='total_catch_weight'
)
    

Here if we fish "too hard" we end up killing off the population before it has a chance to grow to a meaningful size. So we can see that in terms of catch of a single cohort there is a balance between natural mortality and growth rates. This situation obviously further complicates if you include both the cost to fish (which increases linearly with fishing effort and thus mortality under our current assumptions) and the value of particular sizes of fish. 